In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
movies['original_language'].value_counts()

en    4505
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
cn      12
ru      11
ko      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: original_language, dtype: int64

In [5]:
movies.isna().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [6]:
movies[movies['homepage'].isna()].isna().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  625
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [7]:
movies1 = movies['tagline'].dropna()

In [8]:
4803-3959

844

In [9]:
movies1.head()

0                       Enter the World of Pandora.
1    At the end of the world, the adventure begins.
2                             A Plan No One Escapes
3                                   The Legend Ends
4              Lost in our world, found in another.
Name: tagline, dtype: object

In [10]:
movies.shape

(4803, 20)

In [11]:
movies['tagline'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
4798    False
4799    False
4800     True
4801    False
4802     True
Name: tagline, Length: 4803, dtype: bool

In [12]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [13]:
credits.shape

(4803, 4)

In [14]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [15]:
df = movies.merge(right=credits,left_on='id',right_on='movie_id')
df.shape

(4803, 24)

In [16]:
df.isna().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title_x                    0
vote_average               0
vote_count                 0
movie_id                   0
title_y                    0
cast                       0
crew                       0
dtype: int64

In [17]:
df1 = df.copy()
df1.shape

(4803, 24)

In [18]:
df2 = df1.dropna()
df2.isna().sum()

budget                  0
genres                  0
homepage                0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title_x                 0
vote_average            0
vote_count              0
movie_id                0
title_y                 0
cast                    0
crew                    0
dtype: int64

In [19]:
df2.shape

(1493, 24)

In [20]:
df3 = df.drop('homepage',axis = 'columns')
df3.shape

(4803, 23)

In [21]:
df3.isna().sum()

budget                    0
genres                    0
id                        0
keywords                  0
original_language         0
original_title            0
overview                  3
popularity                0
production_companies      0
production_countries      0
release_date              1
revenue                   0
runtime                   2
spoken_languages          0
status                    0
tagline                 844
title_x                   0
vote_average              0
vote_count                0
movie_id                  0
title_y                   0
cast                      0
crew                      0
dtype: int64

In [22]:
nan_values = list(df3.loc[pd.isna(df3["tagline"]), :].index)
len(nan_values)

844

In [23]:
nan_values

[10,
 56,
 66,
 78,
 128,
 166,
 235,
 288,
 317,
 381,
 465,
 474,
 492,
 536,
 540,
 602,
 655,
 660,
 667,
 713,
 738,
 769,
 786,
 796,
 806,
 822,
 824,
 835,
 845,
 859,
 942,
 952,
 971,
 985,
 1010,
 1011,
 1023,
 1028,
 1069,
 1074,
 1087,
 1088,
 1094,
 1099,
 1144,
 1208,
 1253,
 1256,
 1258,
 1261,
 1267,
 1268,
 1277,
 1284,
 1287,
 1288,
 1290,
 1291,
 1295,
 1306,
 1317,
 1321,
 1341,
 1355,
 1357,
 1360,
 1382,
 1387,
 1393,
 1418,
 1421,
 1460,
 1464,
 1475,
 1478,
 1509,
 1511,
 1516,
 1522,
 1524,
 1528,
 1530,
 1541,
 1543,
 1569,
 1579,
 1600,
 1627,
 1658,
 1665,
 1667,
 1673,
 1676,
 1677,
 1679,
 1681,
 1691,
 1698,
 1709,
 1727,
 1729,
 1730,
 1738,
 1741,
 1769,
 1772,
 1776,
 1801,
 1840,
 1872,
 1880,
 1886,
 1895,
 1897,
 1899,
 1912,
 1919,
 1922,
 1932,
 1933,
 1934,
 1935,
 1937,
 1938,
 1941,
 1950,
 1951,
 1954,
 1962,
 1963,
 1965,
 1979,
 1984,
 1985,
 2001,
 2005,
 2006,
 2010,
 2011,
 2032,
 2055,
 2059,
 2065,
 2077,
 2079,
 2089,
 2119,
 2130,
 2

In [24]:
df4 = df3.drop(nan_values,axis = 0)
df4.shape

(3959, 23)

In [25]:
df4.isna().sum()

budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title_x                 0
vote_average            0
vote_count              0
movie_id                0
title_y                 0
cast                    0
crew                    0
dtype: int64

In [26]:
df4.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [27]:
for i in eval(df4.iloc[0]['genres']):
    print(i['name'])

Action
Adventure
Fantasy
Science Fiction


In [28]:
def genres(x):
    return [i['name'] for i in eval(x)]
            

In [29]:
genres(df4.iloc[0]['genres'])

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [30]:
df5 = df4.copy()

In [31]:
gen_list = df5['genres'].apply(genres)
df5['genres'] = gen_list

In [32]:
df5.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure, Fantasy, Action]",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action, Adventure, Crime]",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [33]:
df5.iloc[0]['production_companies']

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [34]:
df5['keywords'].apply(genres)

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4795     [gang, audition, police fake, homeless, actress]
4796    [distrust, garage, identity crisis, time trave...
4798    [united states–mexico barrier, legs, arms, pap...
4799                                                   []
4801                                                   []
Name: keywords, Length: 3959, dtype: object

In [35]:
df5 = df5.reindex()
df5.shape

(3959, 23)

In [36]:
keyword_list = df5['keywords'].apply(genres)
df5['keywords'] = keyword_list

In [37]:
df5.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [38]:
production_companies_list = df5['production_companies'].apply(genres)
df5['production_companies'] = production_companies_list

production_countries_list = df5['production_countries'].apply(genres)
df5['production_countries'] = production_countries_list

In [39]:
df5.iloc[0]

budget                                                          237000000
genres                      [Action, Adventure, Fantasy, Science Fiction]
id                                                                  19995
keywords                [culture clash, future, space war, space colon...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [Ingenious Film Partners, Twentieth Century Fo...
production_countries           [United States of America, United Kingdom]
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                                                             162.0
spoken_languages        [{"iso_639_1":

In [40]:
spoken_languages_list = df5['spoken_languages'].apply(genres)
df5['spoken_languages'] = spoken_languages_list
df5.iloc[0]

budget                                                          237000000
genres                      [Action, Adventure, Fantasy, Science Fiction]
id                                                                  19995
keywords                [culture clash, future, space war, space colon...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [Ingenious Film Partners, Twentieth Century Fo...
production_countries           [United States of America, United Kingdom]
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                                                             162.0
spoken_languages                      

In [41]:
df5.iloc[0]['cast']

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [42]:
cast_list = df5['cast'].apply(genres)
df5['cast'] = cast_list

In [43]:
df5.iloc[0]['cast']

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'Giovanni Ribisi',
 'Joel David Moore',
 'CCH Pounder',
 'Wes Studi',
 'Laz Alonso',
 'Dileep Rao',
 'Matt Gerald',
 'Sean Anthony Moran',
 'Jason Whyte',
 'Scott Lawrence',
 'Kelly Kilgour',
 'James Patrick Pitt',
 'Sean Patrick Murphy',
 'Peter Dillon',
 'Kevin Dorman',
 'Kelson Henderson',
 'David Van Horn',
 'Jacob Tomuri',
 'Michael Blain-Rozgay',
 'Jon Curry',
 'Luke Hawker',
 'Woody Schultz',
 'Peter Mensah',
 'Sonia Yee',
 'Jahnel Curfman',
 'Ilram Choi',
 'Kyla Warren',
 'Lisa Roumain',
 'Debra Wilson',
 'Chris Mala',
 'Taylor Kibby',
 'Jodie Landau',
 'Julie Lamm',
 'Cullen B. Madden',
 'Joseph Brady Madden',
 'Frankie Torres',
 'Austin Wilson',
 'Sara Wilson',
 'Tamica Washington-Miller',
 'Lucy Briant',
 'Nathan Meister',
 'Gerry Blair',
 'Matthew Chamberlain',
 'Paul Yates',
 'Wray Wilson',
 'James Gaylyn',
 'Melvin Leno Clark III',
 'Carvon Futrell',
 'Brandon Jelkes',
 'Mica

In [44]:
df6 = df5.copy()

In [45]:
df6[df6['status'] == 'Rumored']

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
4453,0,"[Comedy, Drama, Thriller]",57294,"[kidnapping, stand-up comedy, dark comedy, nig...",en,Crying with Laughter,Powerfully redemptive and darkly comedic reven...,0.108135,"[Scottish Screen, EM Media, BBC Scotland, Sync...",[United Kingdom],...,[English],Rumored,A Bad Trip Down Memory Lane,Crying with Laughter,7.0,1,57294,Crying with Laughter,"[Stephen McCole, Malcolm Shields, Andrew Neil,...","[{""credit_id"": ""52fe492dc3a36847f818d031"", ""de..."


In [46]:
count = 0
for i in range(len(df6)):
    try:
        if 'Directing' in [i['department'] for i in eval(df6['crew'][i])]:
            count += 1
    except:
        pass
    
count

3444

In [47]:
[i['name'] for i in eval(df6['crew'][0])]

['Stephen E. Rivkin',
 'Rick Carter',
 'Christopher Boyes',
 'Christopher Boyes',
 'Mali Finn',
 'James Horner',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'Andrew Menzies',
 'Jill Brooks',
 'Margery Simkin',
 'Kevin Ishioka',
 'Dick Bernstein',
 'Shannon Mills',
 'Dennie Thorpe',
 'Jana Vance',
 'Deborah Lynn Scott',
 'Jon Landau',
 'Sean Haworth',
 'Kim Sinclair',
 'Kim Sinclair',
 'Richard F. Mays',
 'Laeta Kalogridis',
 'Mayes C. Rubeo',
 'Mauro Fiore',
 'Scott Herbertson',
 'Woody Schultz',
 'Linda DeVetta',
 'Linda DeVetta',
 'Richard Bluck',
 'Simon Bright',
 'Richard Martin',
 'Steve R. Moore',
 'John Refoua',
 'Karl J. Martin',
 'Chiling Lin',
 'Ilram Choi',
 'Steven Quale',
 'Carla Meyer',
 'Nick Bassett',
 'Jill Cormack',
 'Andy McLaren',
 'Terry Notary',
 'Garrett Warren',
 'Jonathan Rothbart',
 'Stefan Dechant',
 'Todd Cherniawsky',
 'Miranda Rivers',
 'Robert Stromberg',
 'John Harding',
 'Roberto De Angelis',
 'Mike Smithso

In [48]:
df6.shape

(3959, 23)

In [49]:
df6.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",...,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],...,[English],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",...,"[Français, English, Español, Italiano, Deutsch]",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],...,[English],Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],...,[English],Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [50]:
directors = []
for i in range(len(df6)):
    first = []
    try:
        for i in eval(df6.iloc[i]['crew']):
            if i['department'] == 'Directing':
                first.append(i['name'])
    except:
        pass
    directors.append(first)
len(directors)

3959

In [51]:
directors[:10]

[['James Cameron'],
 ['Gore Verbinski', 'Pamela Alch', 'Sharron Reynolds', 'Karen Golden'],
 ['Sam Mendes', 'Jayne-Ann Tenggren', 'Nicoletta Mani', 'Susie Jones'],
 ['Christopher Nolan',
  'Steve Gehrke',
  'Matthew Reedy',
  'Sid Karne',
  'Nilo Otero'],
 ['Andrew Stanton',
  'Annie Penn',
  'Tommy Gormley',
  'Ben Dixon',
  'Andrew M. Ward'],
 ['Sam Raimi', 'Trudy Ramirez'],
 ['Byron Howard',
  'Nathan Greno',
  'Rick Moore',
  'Merrick Rustia',
  'Doug Walker',
  'Brian Kesinger',
  'Daniel Hu',
  'John Murrah',
  'Scott Beattie',
  'Cory Rocco Florimonte',
  'Allen Blaisdell',
  'Rob Dressel',
  'Todd Allen Erickson',
  'Terry Moews',
  'Christopher K. Poplin',
  'Wally Schaab',
  'Richard Turner',
  'David Wainstain'],
 ['Joss Whedon'],
 ['David Yates', 'Anna Worley'],
 ['Zack Snyder', 'Kimi Webber', 'Amber Harley']]

In [52]:
for i in eval(df6.iloc[10]['crew']):
    if i['department'] == 'Directing':
        print(i['name'])

Marc Forster


In [53]:
directors[10]

['Marc Forster']

In [54]:
df6.shape

(3959, 23)

In [55]:
len(directors)

3959

In [56]:
df7 = df6.copy()
df7['directors'] = directors
df7.shape

(3959, 24)

In [57]:
df7.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew,directors
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",...,Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",[James Cameron]
1,300000000,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],...,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[Gore Verbinski, Pamela Alch, Sharron Reynolds..."
2,245000000,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",...,Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[Sam Mendes, Jayne-Ann Tenggren, Nicoletta Man..."
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],...,Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[Christopher Nolan, Steve Gehrke, Matthew Reed..."
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],...,Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[Andrew Stanton, Annie Penn, Tommy Gormley, Be..."


In [58]:
for i in range(len(df7)):
    temp = 0
    try:
        for j in df7.iloc[i]['directors']:
            if j == 'Terry Miller':
                temp = 1
        if temp > 0:
            print(df7.iloc[i])
            print('*********************************************************************************************')
    except:
        pass

budget                                                          100000000
genres                                [Action, Thriller, Science Fiction]
id                                                                    280
keywords                [cyborg, shotgun, post-apocalyptic, dystopia, ...
original_language                                                      en
original_title                                 Terminator 2: Judgment Day
overview                Nearly 10 years have passed since Sarah Connor...
popularity                                                      101.74155
production_companies    [Lightstorm Entertainment, Pacific Western, Ca...
production_countries                   [France, United States of America]
release_date                                                   1991-07-01
revenue                                                         520000000
runtime                                                             137.0
spoken_languages                      

In [59]:
vector = TfidfVectorizer()
vector

TfidfVectorizer()

In [60]:
df7.head()

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew,directors
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",...,Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",[James Cameron]
1,300000000,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],...,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[Gore Verbinski, Pamela Alch, Sharron Reynolds..."
2,245000000,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]","[United Kingdom, United States of America]",...,Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[Sam Mendes, Jayne-Ann Tenggren, Nicoletta Man..."
3,250000000,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",[United States of America],...,Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[Christopher Nolan, Steve Gehrke, Matthew Reed..."
4,260000000,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],[United States of America],...,Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[Andrew Stanton, Annie Penn, Tommy Gormley, Be..."


In [61]:
df7.iloc[0]['genres']

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [62]:
" ".join(['Action', 'Adventure', 'Fantasy', 'Science Fiction'])

'Action Adventure Fantasy Science Fiction'

In [63]:
def tojoin(x):
    return ' '.join(x)
tojoin(['Action', 'Adventure', 'Fantasy', 'Science Fiction'])

'Action Adventure Fantasy Science Fiction'

In [64]:
df8 = df7.copy()
df8['genres'] = df8['genres'].apply(tojoin)
df8.iloc[0]

budget                                                          237000000
genres                           Action Adventure Fantasy Science Fiction
id                                                                  19995
keywords                [culture clash, future, space war, space colon...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [Ingenious Film Partners, Twentieth Century Fo...
production_countries           [United States of America, United Kingdom]
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                                                             162.0
spoken_languages                      

In [65]:
df8.isna().sum()

budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title_x                 0
vote_average            0
vote_count              0
movie_id                0
title_y                 0
cast                    0
crew                    0
directors               0
dtype: int64

In [66]:
df8.shape

(3959, 24)

In [67]:
movies_list = df8['title_x'].to_list()
len(movies_list)

3959

In [68]:
df8.to_csv('movies_updated.csv')

In [69]:
vector = TfidfVectorizer()
vector

TfidfVectorizer()

In [70]:
tojoin(df8.iloc[0]['cast'])

'Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez Giovanni Ribisi Joel David Moore CCH Pounder Wes Studi Laz Alonso Dileep Rao Matt Gerald Sean Anthony Moran Jason Whyte Scott Lawrence Kelly Kilgour James Patrick Pitt Sean Patrick Murphy Peter Dillon Kevin Dorman Kelson Henderson David Van Horn Jacob Tomuri Michael Blain-Rozgay Jon Curry Luke Hawker Woody Schultz Peter Mensah Sonia Yee Jahnel Curfman Ilram Choi Kyla Warren Lisa Roumain Debra Wilson Chris Mala Taylor Kibby Jodie Landau Julie Lamm Cullen B. Madden Joseph Brady Madden Frankie Torres Austin Wilson Sara Wilson Tamica Washington-Miller Lucy Briant Nathan Meister Gerry Blair Matthew Chamberlain Paul Yates Wray Wilson James Gaylyn Melvin Leno Clark III Carvon Futrell Brandon Jelkes Micah Moch Hanniyah Muhammad Christopher Nolen Christa Oliver April Marie Thomas Bravita A. Threatt Colin Bleasdale Mike Bodnar Matt Clayton Nicole Dionne Jamie Harrison Allan Henry Anthony Ingruber Ashley Jeffery Dean Kn

In [71]:
tojoin(['James Cameron'])

'James Cameron'

In [72]:
df8.iloc[0]

budget                                                          237000000
genres                           Action Adventure Fantasy Science Fiction
id                                                                  19995
keywords                [culture clash, future, space war, space colon...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [Ingenious Film Partners, Twentieth Century Fo...
production_countries           [United States of America, United Kingdom]
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                                                             162.0
spoken_languages                      

In [73]:
df8['crew'] = df8['crew'].apply(genres)
df8.iloc[0]

budget                                                          237000000
genres                           Action Adventure Fantasy Science Fiction
id                                                                  19995
keywords                [culture clash, future, space war, space colon...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [Ingenious Film Partners, Twentieth Century Fo...
production_countries           [United States of America, United Kingdom]
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                                                             162.0
spoken_languages                      

In [74]:
df8['keywords'] = df8['keywords'].apply(tojoin)

In [75]:
df8['cast'] = df8['cast'].apply(tojoin)
df8['crew'] = df8['crew'].apply(tojoin)

In [76]:
df8.iloc[0]

budget                                                          237000000
genres                           Action Adventure Fantasy Science Fiction
id                                                                  19995
keywords                culture clash future space war space colony so...
original_language                                                      en
original_title                                                     Avatar
overview                In the 22nd century, a paraplegic Marine is di...
popularity                                                     150.437577
production_companies    [Ingenious Film Partners, Twentieth Century Fo...
production_countries           [United States of America, United Kingdom]
release_date                                                   2009-12-10
revenue                                                        2787965087
runtime                                                             162.0
spoken_languages                      

In [77]:
df8['directors'] = df8['directors'].apply(tojoin)

In [78]:
df8.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'movie_id', 'title_y', 'cast', 'crew', 'directors'],
      dtype='object')

In [79]:
df9 = df8.copy()
df9.index = [i for i in range(len(df8))]

In [80]:
req = ['genres', 'keywords', 'tagline', 'cast', 'directors']
req

['genres', 'keywords', 'tagline', 'cast', 'directors']

In [81]:
combined_data = df9['genres'] + ' ' + df9['keywords'] + ' ' + df9['tagline'] + ' ' + df9['cast'] + ' ' + df9['directors']
len(combined_data)

3959

In [82]:
combined_data

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
3954    Drama gang audition police fake homeless actre...
3955    Science Fiction Drama Thriller distrust garage...
3956    Action Crime Thriller united states–mexico bar...
3957    Comedy Romance  A newlywed couple's honeymoon ...
3958      A New Yorker in Shanghai Daniel Henney Eliza...
Length: 3959, dtype: object

In [83]:
features = vector.fit_transform(combined_data)
print(features)

  (0, 5882)	0.04690163945871835
  (0, 30750)	0.061589830954385104
  (0, 18704)	0.08210139725365508
  (0, 40723)	0.09414020085923148
  (0, 26730)	0.09414020085923148
  (0, 39676)	0.09414020085923148
  (0, 30996)	0.030137332359634186
  (0, 2648)	0.05487131486375567
  (0, 38541)	0.08969703664581657
  (0, 916)	0.05566001423998687
  (0, 35404)	0.06375764329058549
  (0, 33551)	0.07545924517438252
  (0, 30942)	0.08969703664581657
  (0, 31726)	0.07450575786149358
  (0, 13752)	0.07135328268274699
  (0, 30170)	0.07765823304024015
  (0, 29234)	0.08210139725365508
  (0, 35548)	0.05012790745021702
  (0, 29011)	0.09414020085923148
  (0, 22040)	0.05668809810563732
  (0, 27975)	0.09414020085923148
  (0, 18877)	0.07068894648039555
  (0, 26964)	0.07894892207490849
  (0, 36704)	0.045917551200829146
  (0, 17114)	0.05105450348265763
  :	:
  (3957, 8266)	0.19012409701415392
  (3957, 5546)	0.31468266840871645
  (3957, 8957)	0.2087324976693578
  (3957, 7753)	0.06152262937377349
  (3957, 17728)	0.0894104462137

In [84]:
similarity = cosine_similarity(features)
similarity

array([[1.        , 0.02641735, 0.01341002, ..., 0.01208849, 0.0057429 ,
        0.01978298],
       [0.02641735, 1.        , 0.01885864, ..., 0.0052108 , 0.01176453,
        0.01496262],
       [0.01341002, 0.01885864, 1.        , ..., 0.01369179, 0.        ,
        0.00845148],
       ...,
       [0.01208849, 0.0052108 , 0.01369179, ..., 1.        , 0.        ,
        0.        ],
       [0.0057429 , 0.01176453, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.01978298, 0.01496262, 0.00845148, ..., 0.        , 0.        ,
        1.        ]])

In [85]:
movies_list

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',
 'TRON: Legacy',
 'C

In [86]:
name = 'Harry potter Prince'
movie_name_crrt = difflib.get_close_matches(name,movies_list)[0]
movie_id = df9[df9['title_x'] == movie_name_crrt].values[0][2]
movie_id

767

In [87]:
df9.iloc[8]

budget                                                          250000000
genres                                           Adventure Fantasy Family
id                                                                    767
keywords                witch magic broom school of witchcraft wizardr...
original_language                                                      en
original_title                     Harry Potter and the Half-Blood Prince
overview                As Harry begins his sixth year at Hogwarts, he...
popularity                                                      98.885637
production_companies                         [Warner Bros., Heyday Films]
production_countries           [United Kingdom, United States of America]
release_date                                                   2009-07-07
revenue                                                         933959197
runtime                                                             153.0
spoken_languages                      

In [88]:
def getindex(given_id):
    for i in range(len(df9)):
        if df9['id'][i] == given_id:
            return i

In [89]:
index_of_the_movie = getindex(movie_id)
index_of_the_movie

8

In [90]:
score_of_similarity = list(enumerate(similarity[index_of_the_movie]))
score_of_similarity

[(0, 0.03605053780669609),
 (1, 0.022330085104219397),
 (2, 0.022706428849393223),
 (3, 0.04583959499430004),
 (4, 0.010761113258514811),
 (5, 0.033175920858850384),
 (6, 0.01878186101274036),
 (7, 0.029413170297254192),
 (8, 1.0000000000000004),
 (9, 0.020762951462516544),
 (10, 0.04282851524392027),
 (11, 0.022918080665705932),
 (12, 0.043443329315564966),
 (13, 0.017648398142973035),
 (14, 0.03947373476722297),
 (15, 0.041070036995201065),
 (16, 0.02979949464236907),
 (17, 0.03309823064242578),
 (18, 0.013962611426062534),
 (19, 0.016536688669502614),
 (20, 0.047392646162733476),
 (21, 0.0127762540587139),
 (22, 0.03930824748507468),
 (23, 0.0411346541964894),
 (24, 0.029350411081753312),
 (25, 0.03636332695137175),
 (26, 0.028033525762156015),
 (27, 0.02530595126164078),
 (28, 0.040066040743953354),
 (29, 0.04858831258521417),
 (30, 0.024945916268048828),
 (31, 0.09267017536082799),
 (32, 0.03528032045843973),
 (33, 0.05021788053611953),
 (34, 0.03557705349258688),
 (35, 0.02275602

In [91]:
similar_movies_list = sorted(score_of_similarity,key=lambda x:x[1],reverse=True)[:12]
similar_movies_list

[(8, 1.0000000000000004),
 (109, 0.4719721695925375),
 (110, 0.4089321363918945),
 (185, 0.38418019833739825),
 (269, 0.35458676390043653),
 (191, 0.30846933989821934),
 (31, 0.09267017536082799),
 (1226, 0.08980813845978523),
 (101, 0.08856690898521619),
 (1690, 0.08176181974149538),
 (1501, 0.08083774414354196),
 (2264, 0.078780968329035)]

In [92]:
for i in similar_movies_list:
    print(df9.iloc[i[0]]['title_x'])

Harry Potter and the Half-Blood Prince
Harry Potter and the Order of the Phoenix
Harry Potter and the Goblet of Fire
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Chamber of Secrets
Harry Potter and the Philosopher's Stone
Alice in Wonderland
Stormbreaker
Alice Through the Looking Glass
Agent Cody Banks 2: Destination London
The Curse of the Were-Rabbit
Becoming Jane


In [93]:
def find_movie(movie_name):
    try:
        movie_name_crrt = difflib.get_close_matches(movie_name,movies_list)[0]
        movie_id = df9[df9['title_x'] == movie_name_crrt].values[0][2]
        index_of_the_movie = getindex(movie_id)
        score_of_similarity = list(enumerate(similarity[index_of_the_movie]))
        similar_movies_list = sorted(score_of_similarity,key=lambda x:x[1],reverse=True)[:12]
        for i in similar_movies_list:
            print(df9.iloc[i[0]]['title_x'] + ' <-----> ' + df9.iloc[i[0]]['directors'])
    except:
        print('No Matches Found!')

In [94]:
find_movie('love')

Beloved <-----> Jonathan Demme
The Soloist <-----> Joe Wright Kerry Lyn McKissick
Y Tu Mamá También <-----> Alfonso Cuarón
For Colored Girls <-----> Tyler Perry
The Color Purple <-----> Steven Spielberg
Bee Movie <-----> Steve Hickner Simon J. Smith
Soul Food <-----> George Tillman, Jr.
Bled <-----> Christopher Hutson
The Manchurian Candidate <-----> Jonathan Demme Arthur Anderson Eva Z. Cabrera
The Rainmaker <-----> Francis Ford Coppola
The Ladykillers <-----> Joel Coen Ethan Coen
Diary of a Mad Black Woman <-----> Darren Grant Gail Hunter


In [95]:
df9.to_csv('Final.csv')

In [96]:
find_movie('Iron man')

Iron Man <-----> Jon Favreau Cristina Weigmann
Iron Man 2 <-----> Jon Favreau
Iron Man 3 <-----> Shane Black
Ant-Man <-----> Peyton Reed Walter Gasparovic Lars P. Winther Kerry Lyn McKissick Brian Avery Galligan
Thor: The Dark World <-----> Alan Taylor
The Avengers <-----> Joss Whedon Trudy Ramirez
Thor <-----> Kenneth Branagh
Captain America: The First Avenger <-----> Joe Johnston
The Incredible Hulk <-----> Louis Leterrier Louis Leterrier
Avengers: Age of Ultron <-----> Joss Whedon
Captain America: The Winter Soldier <-----> Anthony Russo Joe Russo
Captain America: Civil War <-----> Anthony Russo Joe Russo Joe Russo


In [97]:
movies_list[100]

'Poseidon'

In [98]:
find_movie('Batman')

Batman <-----> Tim Burton
Batman Returns <-----> Tim Burton
Batman & Robin <-----> Joel Schumacher William M. Elvin
Batman Forever <-----> Joel Schumacher
Superman III <-----> Richard Lester
Superman IV: The Quest for Peace <-----> Sidney J. Furie
The Dark Knight <-----> Christopher Nolan Steve Gehrke Nilo Otero
Man of Steel <-----> Zack Snyder
Batman: The Dark Knight Returns, Part 2 <-----> Jay Oliva
Batman Begins <-----> Christopher Nolan Annie Penn Cliff Lanning
Superman <-----> Richard Donner
Undercover Brother <-----> Malcolm D. Lee


In [99]:
find_movie('Shipping')

The Shipping News <-----> Lasse Hallström Susanna David
Safe Haven <-----> Lasse Hallström
An Ideal Husband <-----> Oliver Parker
Wayne's World <-----> Penelope Spheeris
Dear John <-----> Lasse Hallström
Nanny McPhee and the Big Bang <-----> Susanna White
Snow Angels <-----> David Gordon Green
Hachi: A Dog's Tale <-----> Lasse Hallström Robin Squibb Cara Giallanza
Horrible Bosses 2 <-----> Sean Anders Jane Goldsmith
Moon <-----> Duncan Jones
Notes on a Scandal <-----> Richard Eyre
What's Eating Gilbert Grape <-----> Lasse Hallström


In [100]:
%store find_movie

Proper storage of interactively declared classes (or instances
of those classes) is not possible! Only instances
of classes in real modules on file system can be %store'd.



In [102]:
find_movie('harry')

Charly <-----> Ralph Nelson
Silent Running <-----> Douglas Trumbull
The Black Hole <-----> Gary Nelson Sandy Nelson
The Tale of Despereaux <-----> Robert Stevenhagen Sam Fell
The Box <-----> Richard Kelly
The Family Stone <-----> Thomas Bezucha
The Land Girls <-----> David Leland
Splice <-----> Vincenzo Natali Lisa Burling
Hollow Man <-----> Paul Verhoeven
Flashdance <-----> Adrian Lyne Joanie Blum
Gossip <-----> Davis Guggenheim
The Island of Dr. Moreau <-----> John Frankenheimer Richard Stanley


In [109]:
df9.iloc[10]

budget                                                          200000000
genres                                    Adventure Action Thriller Crime
id                                                                  10764
keywords                killing undercover secret agent british secret...
original_language                                                      en
original_title                                          Quantum of Solace
overview                Quantum of Solace continues the adventures of ...
popularity                                                     107.928811
production_companies                                    [Eon Productions]
production_countries           [United Kingdom, United States of America]
release_date                                                   2008-10-30
revenue                                                         586090727
runtime                                                             106.0
spoken_languages          [English, Es